In [56]:
%pylab 
%reload_ext autoreload
%autoreload 2


import pandas as pd

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


/home/parawr/anaconda2/envs/pytorch/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['indices']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
# fashion_mnist = pd.read_csv('../features/feat100k.csv')

In [3]:
# fashion_mnist.shape

NameError: name 'fashion_mnist' is not defined

In [7]:
import os 
from natsort import natsorted

names = natsorted(os.listdir('/home/parawr/Projects/clusterFacadeData/superFacade'))
names_to_class = {ii:o for ii, o in enumerate(names)}

names_to_class[10] = 'Switzerlan'

In [8]:
{'Austria': 0,
 'Belgium': 1,
 'China': 2,
 'Denmark': 3,
 'France': 4,
 'Germany': 5,
 'Greece': 6,
 'Italy': 7,
 'Netherlands': 8,
 'Romania': 9,
 'Switzerland': 10,
 'UK': 11,
 'USA_Chicago': 12,
 'USA_Dallas': 13,
 'USA_Hawaii': 14,
 'USA_LosAngeles': 15,
 'USA_NewYork': 16}

{'Austria': 0,
 'Belgium': 1,
 'China': 2,
 'Denmark': 3,
 'France': 4,
 'Germany': 5,
 'Greece': 6,
 'Italy': 7,
 'Netherlands': 8,
 'Romania': 9,
 'Switzerland': 10,
 'UK': 11,
 'USA_Chicago': 12,
 'USA_Dallas': 13,
 'USA_Hawaii': 14,
 'USA_LosAngeles': 15,
 'USA_NewYork': 16}

In [23]:
import pickle 
with open('id_to_name.pickle', 'rb') as fd:
    id_to_name = pickle.load(fd)
    
id_to_class = []
for ii, name in enumerate(id_to_name):
    hit = 0
    for jj, class_name in names_to_class.items():
        if class_name in name:
            hit += 1
            if hit > 1:
                continue
            id_to_class.append(jj)

In [25]:
len(id_to_class)

199723

In [27]:
len(id_to_name)

199723

In [8]:
len(id_to_class)

200410

In [12]:
id_to_name[55555:55560]

['France_Paris_way69859754_Fid2608_LO7W5gi-Z9USyRnvqV3j6Q.jpg',
 'France_Paris_way69859754_Fid2609_LO7W5gi-Z9USyRnvqV3j6Q.jpg',
 'France_Paris_way69859754_Fid2610_LO7W5gi-Z9USyRnvqV3j6Q.jpg',
 'France_Paris_way69859754_Fid2611_LO7W5gi-Z9USyRnvqV3j6Q.jpg',
 'France_Paris_way69859759_Fid19912_zfCohxpwvKGTIt2WbPbjBQ.jpg']

In [ ]:
set(range(len_id)).difference(set(id_to_class))

In [30]:
id_to_name[5725]

'Switzerlan_Bern_way97246574_wall_3_2_n3Jx5nKwFqvde_0OB7l2SA_VP_0_1_FId_749_.jpg'

In [46]:
nrows, ncols = fashion_mnist.shape
nrows, ncols

NameError: name 'nrows' is not defined

In [38]:
id_to_class = id_to_class[:nrows]
len(id_to_class)

7653

In [28]:
import umap
import numpy as np
import pandas as pd
import requests
import os
import datashader as ds
import datashader.utils as utils
import datashader.transfer_functions as tf

import matplotlib.pyplot as plt
import seaborn as sns
import time

sns.set(context="paper", style="white")

# source_df = pd.read_csv('feat7k.csv')

# data = source_df.iloc[:, 1:4095].values.astype(np.float32)
target = id_to_class

In [29]:
pal = [
    '#FFFFFF',
    '#000000',
    '#D70000',
    '#8C3CFF',
    '#028800',
    '#00ACC7',
    '#98FF00',
    '#FF7FD1',
    '#6C004F',
    '#FFA530',
    '#00009D',
    '#867068',
    '#004942',
    '#4F2A00',
    '#00FDCF',
    '#BCB7FF',
    '#95B47A',
    ]
color_key = {str(d):c.lower() for d,c in enumerate(pal)}
color_key

{'0': '#ffffff',
 '1': '#000000',
 '10': '#00009d',
 '11': '#867068',
 '12': '#004942',
 '13': '#4f2a00',
 '14': '#00fdcf',
 '15': '#bcb7ff',
 '16': '#95b47a',
 '2': '#d70000',
 '3': '#8c3cff',
 '4': '#028800',
 '5': '#00acc7',
 '6': '#98ff00',
 '7': '#ff7fd1',
 '8': '#6c004f',
 '9': '#ffa530'}

In [41]:
reducer = umap.UMAP(random_state=42)
start = time.time()
embedding = reducer.fit_transform(data)
print(time.time() - start)
np.savez('embedding4096.npz', embedding)

293.666139125824


In [30]:
embedding = np.load('embedding100k_4096.npz')['arr_0']

In [31]:
embedding.shape

(199723, 2)

In [6]:
np.unique(target)

NameError: name 'target' is not defined

In [32]:
df = pd.DataFrame(embedding, columns=('x', 'y'))
df['class'] = pd.Series([str(x) for x in target], dtype="category")

In [4]:
cvs = ds.Canvas(plot_width=400, plot_height=400)
agg = cvs.points(df, 'x', 'y', ds.count_cat('class'))
img = tf.shade(agg, color_key=color_key, how='eq_hist')


utils.export_image(img, filename='feat7k', background='white')

image = plt.imread('feat7k.png')
fig, ax = plt.subplots(figsize=(6, 6))
plt.imshow(image)
plt.setp(ax, xticks=[], yticks=[])
plt.title("Fashion MNIST data embedded\n"
          "into two dimensions by UMAP\n"
          "visualised with Datashader",
          fontsize=12)

plt.show()

NameError: name 'ds' is not defined

In [ ]:
target

In [57]:
indices = np.random.randint(0, len(target), len(target)//10)
print(len(indices))



fixed_names_to_class = names_to_class
fixed_names_to_class[10] = 'Switzerland'

tab17 = plt.cm.tab20
tab17.colors = tab17.colors[:17]
tab17.N = 17


sc = plt.scatter(embedding[indices, 0], embedding[indices, 1], c=np.array(target)[indices], cmap=tab17, vmin=0, vmax=17, alpha=0.5)
grid('on')
cbar = plt.colorbar(alpha=1.0)

cbar.set_ticks(array(range(17)) + 0.5)
cbar.set_ticklabels(list(names_to_class.values()))


19972


/home/parawr/anaconda2/envs/pytorch/lib/python3.5/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [30]:
tab17.colors[:17]

((0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (0.6823529411764706, 0.7803921568627451, 0.9098039215686274),
 (1.0, 0.4980392156862745, 0.054901960784313725),
 (1.0, 0.7333333333333333, 0.47058823529411764),
 (0.17254901960784313, 0.6274509803921569, 0.17254901960784313),
 (0.596078431372549, 0.8745098039215686, 0.5411764705882353),
 (0.8392156862745098, 0.15294117647058825, 0.1568627450980392),
 (1.0, 0.596078431372549, 0.5882352941176471),
 (0.5803921568627451, 0.403921568627451, 0.7411764705882353),
 (0.7725490196078432, 0.6901960784313725, 0.8352941176470589),
 (0.5490196078431373, 0.33725490196078434, 0.29411764705882354),
 (0.7686274509803922, 0.611764705882353, 0.5803921568627451),
 (0.8901960784313725, 0.4666666666666667, 0.7607843137254902),
 (0.9686274509803922, 0.7137254901960784, 0.8235294117647058),
 (0.4980392156862745, 0.4980392156862745, 0.4980392156862745),
 (0.7803921568627451, 0.7803921568627451, 0.7803921568627451),
 (0.7372549019607844, 0.74117647

In [27]:
names_to_class

{0: 'Austria',
 1: 'Belgium',
 2: 'China',
 3: 'Denmark',
 4: 'France',
 5: 'Germany',
 6: 'Greece',
 7: 'Italy',
 8: 'Netherlands',
 9: 'Romania',
 10: 'Switzerlan',
 11: 'UK',
 12: 'USA_Chicago',
 13: 'USA_Dallas',
 14: 'USA_Hawaii',
 15: 'USA_LosAngeles',
 16: 'USA_NewYork'}

In [47]:
len(list(names_to_class.values())+['x', 'x', 'x'])

20